# Sample - Metadata

In [1]:
NAME = '01-02_sample_sp500_metadata'
PROJECT = 'conference-calls-sentiment'
PYTHON_VERSION = '3.7.0'

### Imports  

In [2]:
import os
import re
import numpy as np
import pandas as pd

# Plotting
import matplotlib.pyplot as plt

# Logging
from utils import log_step

### Settings

In [3]:
workdir = re.sub("(?<={})[\w\W]*".format(PROJECT), "", os.getcwd())
os.chdir(workdir)

pipeline = os.path.join('2_pipeline', NAME)
if not os.path.exists(pipeline):
    os.makedirs(pipeline)
    for folder in ['out', 'store', 'tmp']:
        os.makedirs(os.path.join(pipeline, folder))

---
# Main Code

## Read metadata

In [ ]:
METADATA_PATH = os.path.join('0_data', 'conference_calls_transcripts', 'metadata.csv')
metadata = pd.read_csv(METADATA_PATH, sep='\t', engine='python')
metadata.head()

## Pipeline

In [10]:
@log_step
def start_pipeline(metadata):
    return metadata.copy()

In [11]:
@log_step
def handle_dates(metadata):
    metadata['event_date'] = pd.to_datetime(metadata['event_date'], format=r'%d-%b-%Y')
    metadata['year'] = metadata['event_date'].dt.year
    return metadata

In [12]:
@log_step
def filter_sp500_firms(metadata):
    SP500_PATH = os.path.join('2_pipeline', '01-01_sample_sp500_constituents', 'out', 'sp500_consituents_2000-2020.feather')
    sp500 = pd.read_feather(SP500_PATH)

    metadata_sp500 = metadata.merge(sp500, on='gvkey')

    in_sp500 = metadata_sp500['event_date'].between(metadata_sp500['start_date'], metadata_sp500['end_date'])
    metadata_sp500 = metadata_sp500[in_sp500]
    
    return metadata_sp500

In [13]:
@log_step
def filter_earnings_calls(metadata):
    earning_dates = pd.read_csv(os.path.join('0_data', 'sp500_constituents', 'sp500_quarterly-earnings_2000-2020.csv'))
    
    earning_dates['event_date'] = pd.to_datetime(earning_dates['rdq'], format=r'%Y%m%d')
    earning_dates = earning_dates[['gvkey', 'event_date']].drop_duplicates()

    sp500_earnings = metadata.merge(earning_dates, on=['gvkey', 'event_date'], validate='m:1')

    # Drop duplicates if there are multiple conference calls on one day
    sp500_earnings = sp500_earnings.drop_duplicates(['gvkey', 'event_date']).reset_index()
    return sp500_earnings

In [22]:
def get_transcript_id_mapping(path):
    filepaths = []
    for year in os.listdir(path):
        year_path = os.path.join(path, year)
        if os.path.isdir(year_path):
            filenames = os.listdir(year_path)
            for filename in filenames:
                filepaths.append(os.path.join(year, filename))

    transcript_ids = [int(f.split('-')[-2]) for f in filepaths]
    return {k:v for k, v in zip(transcript_ids, filepaths)}

@log_step
def add_filepaths(metadata):
    TRANSCRIPTS_PATH = os.path.join('0_data', 'conference_calls_transcripts')
    transcript_id_mapping = get_transcript_id_mapping(TRANSCRIPTS_PATH)
    metadata['filepath'] = metadata['transcript_id'].apply(lambda x: transcript_id_mapping.get(x, np.nan))
    return metadata

In [23]:
@log_step
def clean_metadata(metadata):
    columns = ['gvkey', 'coname', 'event_date', 'event_name', 'event_time',
               'event_type', 'transcript_id', 'filepath', 'year',
               'cik', 'ticker', 'regcountrycode']
    
    metadata_clean = (metadata[columns]
                      .copy()
                      .rename(columns={'regcountrycode': 'country'})
                      .assign(gvkey=lambda x: x['gvkey'].astype('int'),
                              year=lambda x: x['year'].astype('int'))
                      .query('2004 <= year <= 2020')
                      .sort_values(['gvkey', 'event_date'])
                      .reset_index(drop=True))
    return metadata_clean

In [24]:
sample_metadata = (metadata.pipe(start_pipeline)
                           .pipe(handle_dates)
                           .pipe(filter_sp500_firms)
                           .pipe(filter_earnings_calls)
                           .pipe(add_filepaths)
                           .pipe(clean_metadata))
sample_metadata.head()

[start_pipeline] nrows=351,851 ncols=32
[handle_dates] nrows=351,851 ncols=33
[filter_sp500_firms] nrows=37,418 ncols=39
[filter_earnings_calls] nrows=29,365 ncols=40
[add_filepaths] nrows=29,365 ncols=41
[clean_metadata] nrows=26,651 ncols=12


,gvkey,coname,event_date,event_name,event_time,event_type,transcript_id,filepath,year,cik,ticker,country
0,1013,ADC TELECOMMUNICATIONS INC,2004-02-18,Q1 2004 ADC Earnings Conference Call,17:00,Earnings Conference Call,137638020605,2004\2004-Feb-18-ADCT.OQ^L10-137638020605-Tran...,2004,61478.0,ADCT,US
1,1013,ADC TELECOMMUNICATIONS INC,2004-05-19,Q2 2004 ADC Earnings Conference Call,17:00,Earnings Conference Call,138206702539,2004\2004-May-19-ADCT.OQ^L10-138206702539-Tran...,2004,61478.0,ADCT,US
2,1013,ADC TELECOMMUNICATIONS INC,2004-08-25,Q3 2004 ADC Earnings Conference Call,17:00,Earnings Conference Call,137720754281,2004\2004-Aug-25-ADCT.OQ^L10-137720754281-Tran...,2004,61478.0,ADCT,US
3,1013,ADC TELECOMMUNICATIONS INC,2004-12-14,Q4 2004 ADC Earnings Conference Call,17:00,Earnings Conference Call,138829529879,2004\2004-Dec-14-ADCT.OQ^L10-138829529879-Tran...,2004,61478.0,ADCT,US
4,1013,ADC TELECOMMUNICATIONS INC,2005-02-28,Q1 2005 ADC Earnings Conference Call,17:00,Earnings Conference Call,139950184137,2005\2005-Feb-28-ADCT.OQ^L10-139950184137-Tran...,2005,61478.0,ADCT,US


In [28]:
sample_metadata.to_feather(os.path.join(pipeline, 'out', 'sample_metadata.feather'))